# Coroutine And Async
> Deep dive into coroutine and async
- toc: true 
- badges: true
- comments: true
- categories: [yield, generator, concurrent]
- hide: true

# Coroutine 101

Coroutine was introduced in Python 3.5 and it's very differnt from normal functions. Coroutine can't be run on its own, it needs to be driven by somebody else.

In [3]:
async def greeting_async(name):
    return 'Hello '+name

In [5]:
def greeting_normal(name):
    return 'Hello '+name

In [6]:
greeting_normal('Peter')

'Hello Peter'

In [16]:
greeting_async('Peter')

<coroutine object greeting_async at 0x0000029FBB35E248>

In order to run a coroutine, you need to send it a value.

In [17]:
g=_

In [18]:
g.send(None)

StopIteration: Hello Peter

Then you get an Stop Iteration Exception. This is because under the hood, coroutine is implemented by generator. With that in mind, you could write a run method to run the coroutine.

In [23]:
def run(coro):
    try:
        coro.send(None)
    except StopIteration as e:
        return e.value

In [24]:
run(greeting_async('Peter'))

'Hello Peter'

If you have one of these coroutines, it can call another coroutine without the need to catch the StopIteration. It turns out you could use the await statement to call another coroutine like a normal function.

In [28]:
async def print_greeting(name):
    print(await greeting_async(name))

In [29]:
run(print_greeting('Guido'))

Hello Guido


Using await, you could call the coroutine as normal functions.

In [30]:
async def print_several_greetings():
    names=['Peter','Guido','Dave']
    for n in names:
        await print_greeting(n)

In [31]:
run(print_several_greetings())

Hello Peter
Hello Guido
Hello Dave


We could think of coroutine as alternate async version of functions comparing to normal functions. You have the alternate way of calling functions using await. Doing this, you could almost everything you would do with normal Python functions.

You could even call coroutine recursively using await and async.

In [36]:
async def fib(n):
    if n<=2:
        return 1
    else:
        return await fib(n-1) + await fib(n-2)

In [37]:
run(fib(10))

55

Rule of thumbs about where to use async and await:

1. You can't use await in interactive REPL. Because the interactive REPL doesn't have the management of coroutine.
2. You can't use await inside a normal function. It will raise a syntax error.
3. You could use it in any Python expression.
4. You could use it in list, dict comprehension (not support in Python 3.5 though).
5. You can't use it in lambda function.
6. You could use it in class instance, class and static method.
7. You can't use in most magic methods
8. You can use it in magic methods about getting something, like @property, \_\_getitem\_\_, \_\_getattr\_\_

In [44]:
async def fib_list(n):
    return [await fib(i) for i in range(n)]

In [45]:
run(fib_list(10))

[1, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [54]:
async def fib_lambda(func,n):
    i=func(n)
    return await(fib(i))

async def add_one(n):
    return n+1

In [60]:
run(fib_lambda(lambda x: await add_one(x),5))

SyntaxError: 'await' outside async function (<ipython-input-60-8bfa3f2bf05f>, line 4)

In [72]:
class Spam: 
    async def instance_method(self):
        print("instance method")
    
    @classmethod
    async def class_method(cls):
        print("class method")
    
    @staticmethod
    async def static_method():
        print("static method")

In [73]:
spam=Spam()

In [74]:
run(spam.instance_method())

instance method


In [75]:
run(Spam.class_method())

class method


In [76]:
run(Spam.static_method())

static method


In [77]:
class Spam:
    async def __init__(self):
        print("init")

In [78]:
spam=Spam()

C:\Users\pyang\AppData\Local\Continuum\anaconda3\envs\pandas\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: coroutine 'Spam.__init__' was never awaited
  """Entry point for launching an IPython kernel.


TypeError: __init__() should return None, not 'coroutine'

In [83]:
class Spam():
    @property
    async def some_value(self):
        return "some_value"

In [84]:
async def main():
    spam=Spam()
    print(await spam.some_value)

In [85]:
run(main())

some_value
